<a href="https://colab.research.google.com/github/AshwinUnnikrishnan/DeepLearning7150/blob/main/LearningProjects/BertReadability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%env WANDB_DISABLED=true


env: WANDB_DISABLED=true


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch
from torch import nn

sns.set()
%matplotlib inline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [5]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [6]:
print(len(train_df))
print(len(test_df))

2834
7


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [9]:
model.classifier = nn.Linear(768,1)
model.num_labels = 1

In [10]:
inputs = tokenizer("Hello, my dog is cute", return_tensors = "pt")
labels = torch.tensor([0]).unsqueeze(0)
inputs['labels'] = labels
outputs = model(**inputs)

In [11]:
inputs

{'input_ids': tensor([[  101,  8667,   117,  1139,  3676,  1110, 10509,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[0]])}

In [13]:
train_datasets = load_dataset('csv', data_files=['train.csv'])
test_datasets = load_dataset('csv', data_files=['test.csv'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2b967f63c0305c9d/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-19f36f648238f188/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
train_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'url_legal', 'license', 'excerpt', 'target', 'standard_error'],
        num_rows: 2834
    })
})

In [15]:
test_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'url_legal', 'license', 'excerpt'],
        num_rows: 7
    })
})

In [16]:
def tokenize_function(examples):
    return tokenizer(examples['excerpt'], padding='max_length', truncation=True, max_length=512)

f_train_datasets = train_datasets.map(tokenize_function, batched=True)
f_train_datasets = f_train_datasets.remove_columns(['id', 'url_legal', 'license', 'excerpt', 'standard_error'])
f_train_datasets = f_train_datasets.rename_column('target', 'labels')
f_train_datasets = f_train_datasets.shuffle(seed=42)

f_test_datasets = test_datasets.map(tokenize_function, batched=True)
f_test_datasets = f_test_datasets.remove_columns(['url_legal', 'license', 'excerpt'])

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
f_train_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2834
    })
})

In [18]:
n_samples = len(f_train_datasets['train'])
n_train = int(0.9 * n_samples)

f_train_dataset = f_train_datasets['train'].select(range(n_train))
f_eval_dataset = f_train_datasets['train'].select(range(n_train, n_samples))

f_test_dataset = f_test_datasets['train']

In [19]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits, labels = logits.squeeze(), labels.squeeze()
    rmse = np.sqrt(np.mean((labels - logits) ** 2))
    return {'RMSE': rmse}

In [20]:
training_args = TrainingArguments(
    'training_args',
    num_train_epochs = 5,
    logging_steps = 200,
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    evaluation_strategy = 'steps'
)

trainer = Trainer(
    model = model,
    train_dataset = f_train_dataset,
    eval_dataset = f_eval_dataset,
    compute_metrics = compute_metrics,
    args = training_args
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [21]:
trainer.train()


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2550
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1595


Step,Training Loss,Validation Loss,Rmse
200,0.891400,0.620205,0.787531
400,0.652900,0.577519,0.759947
600,0.497400,0.624018,0.789948
800,0.525300,0.514302,0.717148
1000,0.461600,0.602658,0.776310
1200,0.386500,0.559289,0.747856
1400,0.390700,0.573934,0.757584


***** Running Evaluation *****
  Num examples = 284
  Batch size = 8
***** Running Evaluation *****
  Num examples = 284
  Batch size = 8
Saving model checkpoint to training_args/checkpoint-500
Configuration saved in training_args/checkpoint-500/config.json
Model weights saved in training_args/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 284
  Batch size = 8
***** Running Evaluation *****
  Num examples = 284
  Batch size = 8
***** Running Evaluation *****
  Num examples = 284
  Batch size = 8
Saving model checkpoint to training_args/checkpoint-1000
Configuration saved in training_args/checkpoint-1000/config.json
Model weights saved in training_args/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 284
  Batch size = 8
***** Running Evaluation *****
  Num examples = 284
  Batch size = 8
Saving model checkpoint to training_args/checkpoint-1500
Configuration saved in training_args/checkpoint-1500/config.json
Model weight

TrainOutput(global_step=1595, training_loss=0.5291820944663499, metrics={'train_runtime': 1322.9564, 'train_samples_per_second': 9.638, 'train_steps_per_second': 1.206, 'total_flos': 3354635835648000.0, 'train_loss': 0.5291820944663499, 'epoch': 5.0})

In [22]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 284
  Batch size = 8


{'eval_loss': 0.5481451749801636,
 'eval_RMSE': 0.7403682470321655,
 'eval_runtime': 8.9666,
 'eval_samples_per_second': 31.673,
 'eval_steps_per_second': 4.015,
 'epoch': 5.0}

In [23]:
pred_output = trainer.predict(f_test_dataset)
pred_targets = pred_output.predictions.squeeze()
pred_ids = f_test_dataset['id']

submission = pd.DataFrame({
    'id': pred_ids,
    'target': pred_targets
})



The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id. If id are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 7
  Batch size = 8


In [24]:
pred_output

PredictionOutput(predictions=array([[-0.24589005],
       [-0.24546266],
       [-0.24557978],
       [-2.5782135 ],
       [-2.02201   ],
       [-0.24580601],
       [-0.24531439]], dtype=float32), label_ids=None, metrics={'test_runtime': 0.2925, 'test_samples_per_second': 23.929, 'test_steps_per_second': 3.418})